In [2]:
# Imports

import matplotlib.pyplot as plt
import numpy as np
import math
import cmath 
import random
import contfrac


In [3]:
def get_permutation_vector(L,l,idx):
  i = 0
  perm = []
  while i < L-l:
    if len(perm) in idx:
        perm = perm + [L-l+idx.index(len(perm))]
    else:
        perm = perm + [i]
        i+=1
  while len(perm) in idx:
      perm = perm + [L-l+idx.index(len(perm))]
  return perm

def apply_u(psi,u,idx):
    l = len(u.shape)//2; L = len(psi.shape)
    print("l="+str(l)+"L="+str(L)+"idx="+str(idx))
    psi = np.tensordot(psi,u,axes=(idx,range(l,2*l)))
    print("shape="+str(psi.shape))
    perm = get_permutation_vector(L,l,idx)
    print("perm="+str(perm))
    psi = psi.transpose(perm)
    return psi

In [4]:
CNOT = np.array([[1,0,0,0],
                 [0,1,0,0],
                 [0,0,0,1],
                 [0,0,1,0]]).reshape([2,2,2,2])


H = 1/np.sqrt(2)*np.array([[1, 1],
                           [1,-1]])



In [5]:
#Construction of the U Matrix
def get_U_matrices(N,x,bitlength_register_A,bitlength_register_B):
    L = bitlength_register_B
    U = np.zeros((2**L,2**L))
    for i in range(2**L):
        if(i<N):
            U[(i*x)%N,i] = 1
        else:
            U[i,i]=1

    Upow2k= np.zeros((2**L,2**L,bitlength_register_A))
    Upow2k[:,:,0]=U[:,:]
    for i in range(1,bitlength_register_A):
        Upow2k[:,:,i]=np.tensordot(Upow2k[:,:,i-1],Upow2k[:,:,i-1],axes=(1,0))
    return Upow2k

def get_CU_matrices(N,x,bitlength_register_A,bitlength_register_B):
    Upow2k = get_U_matrices(N,x,bitlength_register_A,bitlength_register_B)
    L = bitlength_register_B
    CUpow2k=np.zeros((2**(L+1),2**(L+1),bitlength_register_A))
    for i in range(bitlength_register_A):
        CUpow2k[0:2**L,0:2**L,i]=np.identity(2**L)
        CUpow2k[2**L:2**(L+1),2**L:2**(L+1),i]=Upow2k[:,:,i]
    return CUpow2k


In [6]:
def prechecks(number):
    if(number % 2 == 0):
        print("Number is even")
        return False
    if is_prime(number):
        print("Number is already a prime number")
        return False
    (simple_power_boolean,a,b) = simple_power(number) 
    if simple_power_boolean:
        print("N="+str(a)+"^"+str(b))
        return False
    return True
    
def is_prime(number):
    return False

def simple_power(number):
    b_max = math.log(number,3)
    i = 2
    while i < b_max:
        if float(number**(1/i)).is_integer():
            return (True,number**(1/i),i)
        i=i+1
    return (False,-1,-1)

print(prechecks(50))
print(prechecks(13**5))



Number is even
False
True


In [7]:
def get_x(N):
    i = random.randint(2,N)
    while math.gcd(i,N) != 1:
        i = random.randint(2,N)
    return i


In [8]:
# Implementing fourier transform and inverse fourier transform
def R(k): 
    return np.array([[1,0,0,0],
                    [0,1,0,0],
                    [0,0,1,0],
                    [0,0,0,cmath.exp(1j*2*math.pi/(2**k))]]).reshape([2,2,2,2])
def nR(k):
    return np.array([[1,0,0,0],
                    [0,1,0,0],
                    [0,0,1,0],
                    [0,0,0,cmath.exp(-1j*2*math.pi/(2**k))]]).reshape([2,2,2,2])

SWAP = np.array([[1,0,0,0],[0,0,1,0],[0,1,0,0],[0,0,0,1]]).reshape([2,2,2,2])

def FT(psi,idx):
    for i in range(len(idx)):
        psi = apply_u(psi,H,[idx[i]])
        #print("apply: H ["+str(idx[i])+"]")
        k = 2
        for j in range(i+1,len(idx)):
            psi = apply_u(psi,R(k),[idx[j],idx[i]])
        #    print("apply: R("+str(k)+") ["+str(idx[j])+","+str(idx[i])+"]")
            k+=1
    for i in range(len(idx)//2):
        psi = apply_u(psi,SWAP,[idx[i],idx[len(idx)-1-i]])
        #print("apply: SWAP ["+str(idx[i])+","+str(idx[len(idx)-1-i])+"]")
    return psi 

def invFT(psi,idx):
    for i in range(len(idx)//2):
        psi = apply_u(psi,SWAP,[idx[i],idx[len(idx)-1-i]])
        #print("apply: SWAP ["+str(idx[i])+","+str(idx[len(idx)-1-i])+"]")
    for i in range(len(idx)-1,-1,-1):
        k = len(idx)-i
        for j in range(len(idx)-1,i,-1):
            psi = apply_u(psi,nR(k),[idx[j],idx[i]])
        #    print("apply: nR("+str(k)+") ["+str(idx[j])+","+str(idx[i])+"]")
            k-=1
        psi = apply_u(psi,H,[idx[i]])
        #print("apply: H ["+str(idx[i])+"]")
    return psi
        

In [9]:
#Define helper function to analyse the psi state
def totuple(a):
    try:
        return tuple(totuple(i) for i in a)
    except TypeError:
        return a

def measure_subspace_psi(psi,idx):
    ilong = [*range(len(psi.shape))]
    isum = np.setdiff1d(ilong,idx)
    #print(isum)
    abs_psi = np.power(np.absolute(psi),2)
    return np.sum(abs_psi,axis=totuple(isum))

def total_prabability(psi):
    return measure_subspace_psi(psi,[])

def plot_psi_as_barchart(psi):
    L = len(psi.shape)
    mpsi = np.absolute(psi.reshape(2**L))
    y_pos = np.arange(len(mpsi))
    plt.figure(figsize=(15,7))
    plt.bar(y_pos, mpsi,align='center') 
    plt.xticks(y_pos,range(len(mpsi)))
    plt.show()

def plot_psi(psi,subspace_idx):
    mpsi = measure_subspace_psi(psi,subspace_idx).flatten()
    plt.figure(figsize=(15,7))
    plt.plot(range(len(mpsi)),mpsi)
    plt.show()

def print_amplitudes(psi):
  L = len(psi.shape)
  for i in range(2**L):
    X =  np.array([0]*(L - len(list(bin(i)[2:]))) + list(bin(i)[2:]),dtype=int)
    print(X,":",psi.reshape(2**L)[i])



In [ ]:
#Building the Circuit
N = 15
L = len(format(N,'b'))
#print("L="+str(L))
bitlength_register_A = 2*L+3 #(2*L+3)
bitlength_register_B = L
psi = np.zeros(2**(bitlength_register_A+bitlength_register_B),dtype=complex);psi[1]=1


#psi = np.zeros(Bitlength,dtype=complex);psi[4+2+1]=1/sqrt(2);psi[2+1]=1/sqrt(2);
psi = psi.reshape((bitlength_register_A+bitlength_register_B)*[2])
#plot_psi(psi)
#print("Shape:   "+str(psi.shape))
#psi = psi.reshape(L*[2])

#Intializing the first register with H
for i in range(0,bitlength_register_A):
    psi = apply_u(psi,H,[i])



#Applying the CU Matrices
x = 7 #get_x(N)
#print("x="+str(x))

Upow2k = get_CU_matrices(N,x,bitlength_register_A,bitlength_register_B)

for i in range(0,bitlength_register_A):
    #print([bitlength_register_A-1-i]+[*range(bitlength_register_A,bitlength_register_A+bitlength_register_B)])
    psi = apply_u(psi,Upow2k[:,:,i].reshape(2*(bitlength_register_B+1)*[2]),[bitlength_register_A-1-i]+[*range(bitlength_register_A,bitlength_register_A+bitlength_register_B)])
#print_amplitudes(psi)

#Applying the inverse fourier transform
psi = invFT(psi,[*range(bitlength_register_A)])



#print_amplitudes(psi)
plot_psi(psi,[*range(bitlength_register_A)])

 



In [11]:
mpsi = measure_subspace_psi(psi,[*range(bitlength_register_A)]).flatten() 
index = np.argsort(mpsi,axis=0)[-4]
value = (index,2**bitlength_register_A)
print(value)
convergents = list(contfrac.convergents(value))
print(convergents)

def get_largest_convergent_within_range(N,convergents):
    for i in convergents:
        if i[1] > N:
            return j[1]
        j = i

r = get_largest_convergent_within_range(N,convergents)
print(r)

print("gcd(x**r//2-1)="+str(math.gcd(x**(r//2)-1,N)))
print("gcd(x**r//2+1)="+str(math.gcd(x**(r//2)+1,N)))



(1026, 2048)
[(0, 1), (1, 1), (1, 2), (256, 511), (513, 1024)]
2
gcd(x**r//2-1)=5
gcd(x**r//2+1)=3
